In [10]:
import numpy as np
import sympy as sp
import wlcstat.wlc_copoly as wlc_copoly
from scipy.interpolate import interp1d

#FOR VERTEX:
import sys
sys.path.append("./vertex_subroutines")
from GAMcalc import *  # Code for calculating vertex functions
import propagator  # propagator object used to store pre-calculated values
import wignerD as wd # wigner D object used to store pre-calculated values

import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

In [5]:
pset=propagator.prop_set(nlam=10) 
wigset = wd.wigner_d_vals()

In [13]:
sp.var("DEL_A1 DEL_B1 DEL_A2")
DEL_B2 = -DEL_A1 -DEL_B1 -DEL_A2

(DEL_A1, DEL_B1, DEL_A2)

In [7]:
N_1 = 10.
# FA_1_arr = np.arange(0.3, 0.701, .1) #np.array([0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.4, .5, 0.6, 0.7, 0.8]) #np.arange(0.1, 0.701, 0.05) #np.array([0.01, 0.02, 0.05, 0.1, 0.3, .5, 0.7])#
FA_1 = 0.5
PHI_1 = 0.99999

N_2 = N_1
FA_2 = 0.9
PHI_2 = 0.00001

# k_vec = [q_star]#np.logspace(np.log10(k0), np.log10(kf), nk)
# k_ind = 0

# chi_range = np.arange(1, 100, 0.01)/N_1
CHI = 5/N_1


monodisp_FA_arr = np.arange(0.3, 0.701, 0.1)

if (PHI_1 + PHI_2 > 1.00001) or (PHI_1 + PHI_2 < 1 - .00001):
    raise Exception("Phis dont add up to 1")

#### (Choosing k*) -> WRONG for polydisp b/c gamma2 that its based on is monodisp

In [8]:
KS = spinodal(pset, N_1, FA_1) #TODO: WRONG b/c based on monodisperse gamma2 result .need to redo for polydisp
q_star = KS[0]

cyl_q1 = q_star*np.array([1, 0, 0])
cyl_q2 = 0.5*q_star*np.array([-1, np.sqrt(3), 0])
cyl_q3 = 0.5*q_star*np.array([-1, -np.sqrt(3), 0])
cyl_qs = np.array([cyl_q1, cyl_q2, cyl_q3])

# Monodisperse result

### Vertex codebase

In [9]:
gamma3(pset, N_1, FA_1, cyl_qs)

./vertex_subroutines\MultiPoint.py:206: RuntimeWarning: invalid value encountered in cdouble_scalars
  if abs(out/(sum2+tol))<tol and l2>max(lam0_2,lam_2):
./vertex_subroutines\MultiPoint.py:209: RuntimeWarning: invalid value encountered in cdouble_scalars
  if abs(sum2/(sum1+tol))<tol and l1>max(lam0_1,lam_1):


7.703719777548943e-32j

### Matrix

In [19]:
#2x2x2

# Polydisperse result

### Annealed disorder avg

### Exact, expanded matrix

In [22]:
#can construct 4x4x4/ extend einstein notation to now sum over all olymers divided by vol frac. 
#that would give me the pre-incompress result.  BUT HOW TO APPLY INCOMPR. 
#Don t need to actually worry about constructing the matrix. Just need a sum over the appropriate terms
#incompressibility is only applied to the last polymer DelB

#SOLN?:
#literally the same einstein sum, with a for loop before for all polymer types, and including a product of del vectors evaluated 
#at each index within the sum terms
#would have the DEL vector's last entry be the negative sum of all previous entries. 
#can then just procede normally thorugh the sum, now with a set of del vectors products, taking incomp into account

#TO determine:
#WAY to utilize sympy and scipyoptimize? have amlpitudes as symbols in sympy which are then optimized over in scipyoprimize
poly_list = [[FA_1, N_1, PHI_1], [FA_2, N_2, PHI_2]]
dlta = np.array([DEL_A1, DEL_B1, DEL_A2, DEL_B2])

def gamma3binary(pset, Ks, poly_list, dlta):
    #Assuming same N
    #hardcoded DELTA
    #need to return an obe=ejc tthat has 3 different amplitude variables which can be minimized over in scipy
    
    K1, K2, K3 = Ks
    if norm(K1+K2+K3) >= 1e-10:
        raise('Qs must add up to zero')
        
    if not (abs(norm(K1)-norm(K2)) < 1e-5 \
        and abs(norm(K2)-norm(K3)) < 1e-5):
        raise('Qs must have same length')
        
    val = 0
    for i, poly in enumerate(poly_list):
        FA = poly[0]
        N = poly[1]
        phi_poly = poly[2]
        
        s3 = s3wlc(pset, N, FA, Ks)
        s2inv = s2inverse(pset, N, FA, norm(K1))
        for I0, I1, I2 in product([0,1], repeat=3):
            val -= (1/phi_poly) * s3[I0][I1][I2]* (s2inv[I0][0] - s2inv[I0][1])*\
                                   (s2inv[I1][0] - s2inv[I1][1])*\
                                   (s2inv[I2][0] - s2inv[I2][1])*\
                                    -1**(I0 + I1 + I2)*\
                                    dlta[I0+i*2]*dlta[I1+i*2]*dlta[I2+i*2] 

    return val*(N**2) 

In [67]:
G3 = gamma3binary(pset, cyl_qs, poly_list, dlta)

In [50]:
sp.var("x y")

(x, y)

In [51]:
def testrr(coef):
    return coef*x + coef*y

In [52]:
type(testrr(5))

sympy.core.add.Add

In [53]:
func

5*x

In [55]:
func = testrr(5)
funker = sp.lambdify([x, y], func, 'numpy')
funker(5, 2)

35

In [ ]:
mini = optimize.fmin(lambda)

In [ ]:
wlc_copoly.s2_wlc_diblock(k_vec, N_1, FA_1, dimensions=3, alpha_max=25)


In [ ]:
4x4x4, incompres -> 3x3x3. What would I do then? For the Now have at minimum 3 distinct order parameters, 
each with their own amplitude and lengthscale.
Answer: Mus tnow calulcate free energies (amplitude minimization procedure), and can compare in limiting case to monodisperse

In [62]:
G3

<function _lambdifygenerated(DEL_A1, DEL_B1, DEL_A2)>

In [77]:
G3lambd([1,2,1])

TypeError: _lambdifygenerated() missing 2 required positional arguments: 'DEL_B1' and 'DEL_A2'

## Free energy analysis

In [79]:
G3lambd = sp.lambdify([DEL_A1, DEL_B1, DEL_A2], G3, 'numpy')

amp = optimize.fmin(lambda amp: G3lambd(amp[0], amp[1], amp[2]), [1,1,1], disp=False)

C:\Users\Thomas\anaconda3\lib\site-packages\scipy\optimize\optimize.py:689: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[k] = func(sim[k])
C:\Users\Thomas\anaconda3\lib\site-packages\scipy\optimize\optimize.py:721: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxr
C:\Users\Thomas\anaconda3\lib\site-packages\scipy\optimize\optimize.py:714: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxe


In [80]:
amp

array([-4.68841183e+52,  1.09967632e+52, -7.56899531e+52])

In [ ]:
# TO DO: Figure out lambdify, do optiization procedure, extend to G4
    